CLTV CALCULATION

CLTV = (customer value/churn rate)* profit margin

customer value = average order value * purchase frequency

average order value = total revenues / total number of orders

purchase frequency = total number of orders / total number of customers

churn rate = 1- repeat rate (# of customers who will not purchase again)

repeat rate = # of customers who will return

In [25]:
import pandas as pd
pd.set_option("float_format", lambda x: "%.2f" % x)
pd.set_option("display.max_columns", None)
df_ = pd.read_excel("../input/uci-online-retail-ii-data-set/online_retail_II.xlsx",sheet_name="Year 2010-2011")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


In [26]:
# DATA PREPARATION

df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[(df["Quantity"]>0)]
df.dropna(inplace=True)
df["TotalPrice"] = df["Price"] * df["Quantity"]

In [27]:
cltv_df = df.groupby("Customer ID").agg({"Invoice": lambda x: len(x),
                                         "Quantity": lambda x: x.sum(),
                                         "TotalPrice": lambda x: x.sum()})

cltv_df.columns=["total_transaction", "total_unit", "total_price"]

cltv_df.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00,1,74215,77183.60
12347.00,182,2458,4310.00
12348.00,31,2341,1797.24
12349.00,73,631,1757.55
12350.00,17,197,334.40


In [28]:
# 1. Calculate average order value
#   - average order value = total revenues / total number of orders

cltv_df["avg_order_value"] = cltv_df["total_price"] / cltv_df["total_transaction"]

In [29]:
# 2. Calculate purchase frequency
# purchase frequency = total number of orders / total number of customers

cltv_df.shape[0]
# 4339 - total number of customers

cltv_df["purchase_frequency"] = cltv_df["total_transaction"] / 4339
# we standardized the calculation related to general data


In [30]:
# 3. Calculate repeat rate and churn rate
# churn rate = 1 - repeat rate
# remember: repeat rate gives the number of customers who shop at least once divided by the total number of customers

repeat_rate = cltv_df[cltv_df["total_transaction"]>1].shape[0] / cltv_df.shape[0]
churn_rate = 1- repeat_rate

In [31]:
# 4. Calculate profit margin
# assuming we profit from every customer by 5%.
cltv_df["profit_margin"] = cltv_df["total_price"] * 0.05


In [32]:
# 5. Calculate customer lifetime value
# customer value=average order value * purchase frequency

cltv_df["CV"] = cltv_df["avg_order_value"] * cltv_df["purchase_frequency"]

# cltv = (customer value/churn rate)* profit margin

cltv_df["CLTV"] = (cltv_df["CV"] / churn_rate) * cltv_df["profit_margin"]

cltv_df.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV
Customer ID,,,,,,,,
12346.00,1,74215,77183.60,77183.60,0.00,3859.18,17.79,4137019.52
12347.00,182,2458,4310.00,23.68,0.04,215.50,0.99,12900.07
12348.00,31,2341,1797.24,57.98,0.01,89.86,0.41,2243.11
12349.00,73,631,1757.55,24.08,0.02,87.88,0.41,2145.13
12350.00,17,197,334.40,19.67,0.00,16.72,0.08,77.66


In [33]:
cltv_df.sort_values("CLTV", ascending=False)

# However we cannot commend just by looking at the CLTV row values
# We need to transform

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV
Customer ID,,,,,,,,
14646.00,2080,197491,280206.02,134.71,0.48,14010.30,64.58,54524592.81
18102.00,431,64124,259657.30,602.45,0.10,12982.86,59.84,46820773.22
17450.00,337,69993,194550.79,577.30,0.08,9727.54,44.84,26284729.09
16446.00,3,80997,168472.50,56157.50,0.00,8423.62,38.83,19710405.04
14911.00,5677,80515,143825.06,25.33,1.31,7191.25,33.15,14365033.25
...,...,...,...,...,...,...,...,...
17956.00,1,1,12.75,12.75,0.00,0.64,0.00,0.11
16454.00,2,3,6.90,3.45,0.00,0.35,0.00,0.03
14792.00,2,2,6.20,3.10,0.00,0.31,0.00,0.03


In [34]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,100))
scaler.fit(cltv_df[["CLTV"]])
cltv_df["SCALED_CLTV"] = scaler.transform(cltv_df[["CLTV"]])

cltv_df.sort_values("SCALED_CLTV", ascending=False)

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,CV,CLTV,SCALED_CLTV
Customer ID,,,,,,,,,
14646.00,2080,197491,280206.02,134.71,0.48,14010.30,64.58,54524592.81,100.00
18102.00,431,64124,259657.30,602.45,0.10,12982.86,59.84,46820773.22,85.87
17450.00,337,69993,194550.79,577.30,0.08,9727.54,44.84,26284729.09,48.21
16446.00,3,80997,168472.50,56157.50,0.00,8423.62,38.83,19710405.04,36.15
14911.00,5677,80515,143825.06,25.33,1.31,7191.25,33.15,14365033.25,26.35
...,...,...,...,...,...,...,...,...,...
17956.00,1,1,12.75,12.75,0.00,0.64,0.00,0.11,0.00
16454.00,2,3,6.90,3.45,0.00,0.35,0.00,0.03,0.00
14792.00,2,2,6.20,3.10,0.00,0.31,0.00,0.03,0.00


In [35]:
cltv_df[['total_transaction', 'total_unit', 'total_price','SCALED_CLTV']].sort_values("SCALED_CLTV", ascending=False).head(15)

cltv_df1 = cltv_df.reset_index()

cltv_df2=cltv_df1.copy()
cltv_df2["segment"] = pd.qcut(cltv_df2["SCALED_CLTV"], 4, labels=["0_25", "26_50", "51_75", "76_100"])
cltv_df2

cltv_df2.groupby("segment")[["total_transaction", "total_unit",
                             "total_price","CLTV","SCALED_CLTV"]].agg(["mean", "sum"])


total_transaction                total_unit                   \
                    count   mean     sum      count    mean      sum   
segment                                                                
0_25                 1085  17.22   18682       1085  129.49   140493   
26_50                1085  36.98   40122       1085  286.30   310632   
51_75                1084  76.44   82856       1084  649.44   703995   
76_100               1085 236.19  256265       1085 3711.13  4026577   

        total_price                     CLTV                         \
              count    mean        sum count      mean          sum   
segment                                                               
0_25           1085  178.92  194130.09  1085     25.91     28107.00   
26_50          1085  464.37  503836.85  1085    158.18    171620.23   
51_75          1084 1071.88 1161913.31  1084    853.08    924739.47   
76_100         1085 6499.12 7051545.65  1085 234987.18 254961091.91   

        SCALED_CLTV              
              count mean    sum  
segment                          
0_25           1085 0.00   0.05  
26_50          1085 0.00   0.31  
51_75          1084 0.00   1.70  
76_100         1085 0.43 467.61